# Zadatak

Na adresi https://www.foi.unizg.hr/hr/dokumenti nalazi se baza dokumenata Fakulteta organizacije i informatike. Većina dokumenata zadana je u PDF formatu, no neki od dokumenata su čitki (vektorski format) dok su drugi skenirani (rasterski format). Potrebno je:
1) Implementirati program koji će skinuti sve PDF dokumente na lokalno računalo (ili Google Colab / Drive direktorij)
2) Kreirati bazu podataka (SQLite) koja će sadržavati jednu tablicu "dokument". Tablica treba sadržavati šifru dokumenta (autonumber), naslov dokumenta, putanju datoteke (path i filename na lokalnom računalu), URL adresu s koje je dokument skinut, datum, te tekstualni sadržaj dokumenta.
3) Posebno za zadnje polje u tablici potrebno je svaki dokument učitati i ekstrahirati tekst (primjerice putem Python modula PDFMiner - https://github.com/pdfminer/pdfminer.six). Ukoliko je dokument skeniran potrebno je koristiti odgovarajući OCR modul za ekstrakciju teksta (npr. PyTesseract https://pypi.org/project/pytesseract/).
4) Omogućiti pristup skupu podataka putem REST API-ja koji omogućuje isključivo pretraživanje podataka putem:(a) pretraživanja ključnih riječi u tekstu (npr. boolean search), (b) pretraživanje ključnih riječi u naslovu (c) pregled prema datumu (npr. dokumenti od DATUM do DATUM), (d) izlistavanje svih dokumenata.

# Prikupljanje podataka

In [1]:
!mkdir -p data/pdfs data/dummy_data

In [2]:
# General
import os
import sys
import csv

# Scraping
import requests
from bs4 import BeautifulSoup
from requests_html import HTMLSession

# Data manipulation and processing
import numpy as np
import pandas as pd

In [3]:
s = HTMLSession()

def get_data(url):
    response = s.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [4]:
url = 'https://www.foi.unizg.hr/hr/dokumenti'

In [5]:
response = requests.get(url)

In [6]:
response

<Response [200]>

In [7]:
soup = BeautifulSoup(response.text, 'html')

In [8]:
# print(soup.prettify())

## Downloading PDF

In [9]:
def download_pdf(url, path):
    '''
        Napravi download dokumenta sa url adrese u dani direktorij (url: /sites/default/files/[ime_dokumenta].pdf)
    '''
    try:
        response = s.get(url)
        response.raise_for_status()  # Provjerava HTTP greške

        file_name = url.split('/')[-1]
        file_path = os.path.join(path, file_name)

        with open(file_path, 'wb') as f:
            f.write(response.content)

        print(f"Success: {file_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

## Creating a CSV data entry/extracting useful fields

In [10]:
url = 'https://www.foi.unizg.hr/hr/dokument/izmjenjena-odluka-pok-2023'

In [12]:
soup = get_data(url)

In [13]:
link1 = soup.find('a', href=True, type=lambda value: value and 'application/pdf' in value)['href']
datum = soup.find('div', class_ = 'datum').text

In [14]:
fields = soup.find_all('div', class_ = 'field-item even')

for i in range(len(fields)):
    fields[i] = fields[i].text.strip()

In [15]:
ime_datoteke = fields[0]
vrsta_dokumenta = fields[1]
kategorija_dokumenta = fields[2]
datum = datum.strip().split('Kreirano: ')[1]

In [16]:
data = ime_datoteke, link1, datum, vrsta_dokumenta, kategorija_dokumenta
data

('odluka_o_izmjeni_odluke_pok_2023_10_12.pdf',
 'https://www.foi.unizg.hr/sites/default/files/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf',
 '18.12.2023',
 'Odluka',
 'Kvaliteta')

In [17]:
def get_metadata(url):
    '''
        Dohvaca meta podatke za zapis sa dane url adrese (/hr/dokument/[ime_dokumenta])
    '''

    soup = get_data(url)

    try:
        link1 = soup.find('a', href=True, type=lambda value: value and 'application/pdf' in value)['href']
    except:
        link1 = None
    
    datum = soup.find('div', class_ = 'datum').text
    datum = datum.strip().split('Kreirano: ')[1]

    fields = soup.find_all('div', class_ = 'field-item even')
    for i in range(len(fields)):
        fields[i] = fields[i].text.strip()

    
    ime_datoteke = fields[0]

    # U slucaju da nema vrste i kategorije dokumenta
    try:
        vrsta_dokumenta = fields[1]
        kategorija_dokumenta = fields[2]
    except:
        vrsta_dokumenta = None
        kategorija_dokumenta = None

    return ime_datoteke, link1, datum, vrsta_dokumenta, kategorija_dokumenta

In [18]:
metadata = get_metadata('https://www.foi.unizg.hr/hr/dokument/izmjenjena-odluka-pok-2023')
metadata

('odluka_o_izmjeni_odluke_pok_2023_10_12.pdf',
 'https://www.foi.unizg.hr/sites/default/files/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf',
 '18.12.2023',
 'Odluka',
 'Kvaliteta')

In [25]:
metadata = get_metadata('https://www.foi.unizg.hr/hr/dokument/posebni-dio-financijskog-plana-2024-2026')
metadata

('privitak_1b_-_posebni_dio_financijskog_plana_2024-2026_konacan.xlsx',
 None,
 '12.12.2023',
 'Financijski plan',
 'FOI')

## Handling pagination and going through document on the main pages

In [26]:
root_url = 'https://www.foi.unizg.hr/'

In [27]:
url = root_url + '/hr/dokumenti?page=0'

soup = get_data(url)

In [28]:
doc_spans = soup.find_all('span', class_ = 'field-content')
len(doc_spans)

10

In [29]:
doc_spans;

In [30]:
def extract_links(span_list):
    links = []

    for span in doc_spans:
        links_in_span = span.find_all('a', href = True)
        lnk = [l['href'] for l in links_in_span]
        lnk.append(span.text)
        links.append(tuple(lnk))
  
    return links

In [31]:
extract_links(doc_spans)

[('https://www.foi.unizg.hr/sites/default/files/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf',
  '/hr/dokument/izmjenjena-odluka-pok-2023',
  'Izmjenjena odluka POK-a 2023.'),
 ('https://www.foi.unizg.hr/sites/default/files/odluka-izvanredni_izbori_za_studentski_zbor-sijecanj_2024.pdf',
  '/hr/dokument/odluka-o-raspisivanju-izvanrednih-izbora-studentskog-zbora-suzgfoi',
  'Odluka o raspisivanju izvanrednih izbora Studentskog zbora SUZGFOI'),
 ('https://www.foi.unizg.hr/sites/default/files/privitak_1b_-_posebni_dio_financijskog_plana_2024-2026_konacan.xlsx',
  '/hr/dokument/posebni-dio-financijskog-plana-2024-2026',
  'Posebni dio financijskog plana 2024-2026'),
 ('https://www.foi.unizg.hr/sites/default/files/privitak_1_-_opci_dio_prijedlog_financijskog_plana_2024-2026_konacan.xlsx',
  '/hr/dokument/opci-dio-prijedlog-financijskog-plana-2024-2026',
  'Opći dio - prijedlog financijskog plana 2024-2026'),
 ('https://www.foi.unizg.hr/sites/default/files/obrazlozenje_posebnog_dijela_financi

In [32]:
def extract_hr_dokument_links(span_list):
    return [span.find('a', href = True, target = False)['href'] for span in doc_spans]

In [33]:
def main(url):
    '''
        Prolazi kroz linkove glavne stranice, ekstrahira /hr/dokument/ linkove,
        dohvaca meta podatke za svaki dokument i skida pdf dokumente lokalno
    '''
    root_url = 'https://www.foi.unizg.hr/'
    
    soup = get_data(url)
    spans = soup.find_all('span', class_ = 'field-content')

    links = extract_links(spans)

    metadata = [get_metadata(root_url + link[1]) for link in links]

    df1 = pd.DataFrame(metadata, columns = ['naziv_datoteke', 'pdf_link', 'datum', 'vrsta_dokumenta', 'kategorija_dokumenta'])
    df2 = pd.DataFrame(links, columns = ['file_link', 'metadata_link', 'naslov_dokumenta'])

    return pd.concat([df1, df2], axis = 1)

In [34]:
df = main('https://www.foi.unizg.hr/hr/dokumenti')
df

,naziv_datoteke,pdf_link,datum,vrsta_dokumenta,kategorija_dokumenta,file_link,metadata_link,naslov_dokumenta
0,odluka_o_izmjeni_odluke_pok_2023_10_12.pdf,https://www.foi.unizg.hr/sites/default/files/o...,18.12.2023,Odluka,Kvaliteta,https://www.foi.unizg.hr/sites/default/files/o...,/hr/dokument/izmjenjena-odluka-pok-2023,Izmjenjena odluka POK-a 2023.
1,odluka-izvanredni_izbori_za_studentski_zbor-si...,https://www.foi.unizg.hr/sites/default/files/o...,12.12.2023,Odluka,Studentski zbor,https://www.foi.unizg.hr/sites/default/files/o...,/hr/dokument/odluka-o-raspisivanju-izvanrednih...,Odluka o raspisivanju izvanrednih izbora Stude...
2,privitak_1b_-_posebni_dio_financijskog_plana_2...,None,12.12.2023,Financijski plan,FOI,https://www.foi.unizg.hr/sites/default/files/p...,/hr/dokument/posebni-dio-financijskog-plana-20...,Posebni dio financijskog plana 2024-2026
3,privitak_1_-_opci_dio_prijedlog_financijskog_p...,None,12.12.2023,Financijski plan,FOI,https://www.foi.unizg.hr/sites/default/files/p...,/hr/dokument/opci-dio-prijedlog-financijskog-p...,Opći dio - prijedlog financijskog plana 2024-2026
4,obrazlozenje_posebnog_dijela_financijskog_plan...,None,12.12.2023,Financijski plan,FOI,https://www.foi.unizg.hr/sites/default/files/o...,/hr/dokument/obrazlozenje-posebnog-dijela-fina...,Obrazloženje posebnog dijela financijskog plan...
5,obrazlozenje_opceg_dijela_financijskog_plana_2...,None,12.12.2023,Financijski plan,FOI,https://www.foi.unizg.hr/sites/default/files/o...,/hr/dokument/obrazlozenje-opceg-dijela-financi...,Obrazloženje općeg dijela financijskog plana 2...
6,fv_dnevni_red_2023-12-07_3._sjednica.pdf,https://www.foi.unizg.hr/sites/default/files/f...,07.12.2023,Dnevni red,Fakultetsko vijeće,https://www.foi.unizg.hr/sites/default/files/f...,/hr/dokument/dnevni-red-fakultetskog-vijeca-07...,Dnevni red Fakultetskog vijeća 07.12.2023.
7,politika-privatnosti-foi.pdf,https://www.foi.unizg.hr/sites/default/files/p...,24.11.2023,None,None,https://www.foi.unizg.hr/sites/default/files/p...,/hr/dokument/politika-privatnosti-foi-ja,Politika privatnosti FOI-ja
8,popis_gs_u_sukobu_interesa_2023.pdf,https://www.foi.unizg.hr/sites/default/files/p...,20.11.2023,Odluka,Javna nabava,https://www.foi.unizg.hr/sites/default/files/p...,/hr/dokument/popis-gospodarskih-subjekata-s-ko...,Popis gospodarskih subjekata s kojima se ne sm...
9,fv_zakljucci_2023-11-16_2._sjednica.pdf,https://www.foi.unizg.hr/sites/default/files/f...,16.11.2023,Zaključak,Fakultetsko vijeće,https://www.foi.unizg.hr/sites/default/files/f...,/hr/dokument/zakljucak-fakultetskog-vijeca-161...,Zaključak Fakultetskog vijeća 16.11.2023.


### Handling pagination

In [37]:
soup = get_data('https://www.foi.unizg.hr/hr/dokumenti')

In [38]:
page = soup.find('ul', class_ = 'pager')
page

<ul class="pager"><li class="pager-current first">1</li>
<li class="pager-item"><a href="/hr/dokumenti?page=1" title="Idi na stranicu 2">2</a></li>
<li class="pager-item"><a href="/hr/dokumenti?page=2" title="Idi na stranicu 3">3</a></li>
<li class="pager-item"><a href="/hr/dokumenti?page=3" title="Idi na stranicu 4">4</a></li>
<li class="pager-item"><a href="/hr/dokumenti?page=4" title="Idi na stranicu 5">5</a></li>
<li class="pager-ellipsis">…</li>
<li class="pager-next"><a href="/hr/dokumenti?page=1">&gt;&gt;</a></li>
<li class="pager-last last"><a href="/hr/dokumenti?page=93">posljednja &gt;&gt;</a></li>
</ul>

In [65]:
next = page.find('li', class_ = 'pager-next')
'https://www.foi.unizg.hr/hr/dokumenti' + str(next.find('a')['href'])

'https://www.foi.unizg.hr/hr/dokumenti/hr/dokumenti?page=1'

In [66]:
def get_next_page(soup):
    root_url = 'https://www.foi.unizg.hr'
    page = soup.find('ul', class_ = 'pager')
    next_btn = page.find('li', class_ = 'pager-next')
    if next_btn:
        url = root_url + str(next_btn.find('a')['href'])
        return url
    else:
        return None

In [67]:
url1 = 'https://www.foi.unizg.hr/hr/dokumenti'

while True:
    soup = get_data(url1)
    url1 = get_next_page(soup)
    if not url1:
        break

    print(url1)

https://www.foi.unizg.hr/hr/dokumenti?page=1
https://www.foi.unizg.hr/hr/dokumenti?page=2
https://www.foi.unizg.hr/hr/dokumenti?page=3
https://www.foi.unizg.hr/hr/dokumenti?page=4
https://www.foi.unizg.hr/hr/dokumenti?page=5
https://www.foi.unizg.hr/hr/dokumenti?page=6
https://www.foi.unizg.hr/hr/dokumenti?page=7
https://www.foi.unizg.hr/hr/dokumenti?page=8
https://www.foi.unizg.hr/hr/dokumenti?page=9
https://www.foi.unizg.hr/hr/dokumenti?page=10
https://www.foi.unizg.hr/hr/dokumenti?page=11
https://www.foi.unizg.hr/hr/dokumenti?page=12
https://www.foi.unizg.hr/hr/dokumenti?page=13
https://www.foi.unizg.hr/hr/dokumenti?page=14
https://www.foi.unizg.hr/hr/dokumenti?page=15
https://www.foi.unizg.hr/hr/dokumenti?page=16
https://www.foi.unizg.hr/hr/dokumenti?page=17
https://www.foi.unizg.hr/hr/dokumenti?page=18
https://www.foi.unizg.hr/hr/dokumenti?page=19
https://www.foi.unizg.hr/hr/dokumenti?page=20
https://www.foi.unizg.hr/hr/dokumenti?page=21
https://www.foi.unizg.hr/hr/dokumenti?page=

## Downloading in batches

In [35]:
df_test = df.dropna(subset = ['pdf_link'])
len(df_test)

6

In [36]:
df_test

,naziv_datoteke,pdf_link,datum,vrsta_dokumenta,kategorija_dokumenta,file_link,metadata_link,naslov_dokumenta
0,odluka_o_izmjeni_odluke_pok_2023_10_12.pdf,https://www.foi.unizg.hr/sites/default/files/o...,18.12.2023,Odluka,Kvaliteta,https://www.foi.unizg.hr/sites/default/files/o...,/hr/dokument/izmjenjena-odluka-pok-2023,Izmjenjena odluka POK-a 2023.
1,odluka-izvanredni_izbori_za_studentski_zbor-si...,https://www.foi.unizg.hr/sites/default/files/o...,12.12.2023,Odluka,Studentski zbor,https://www.foi.unizg.hr/sites/default/files/o...,/hr/dokument/odluka-o-raspisivanju-izvanrednih...,Odluka o raspisivanju izvanrednih izbora Stude...
6,fv_dnevni_red_2023-12-07_3._sjednica.pdf,https://www.foi.unizg.hr/sites/default/files/f...,07.12.2023,Dnevni red,Fakultetsko vijeće,https://www.foi.unizg.hr/sites/default/files/f...,/hr/dokument/dnevni-red-fakultetskog-vijeca-07...,Dnevni red Fakultetskog vijeća 07.12.2023.
7,politika-privatnosti-foi.pdf,https://www.foi.unizg.hr/sites/default/files/p...,24.11.2023,None,None,https://www.foi.unizg.hr/sites/default/files/p...,/hr/dokument/politika-privatnosti-foi-ja,Politika privatnosti FOI-ja
8,popis_gs_u_sukobu_interesa_2023.pdf,https://www.foi.unizg.hr/sites/default/files/p...,20.11.2023,Odluka,Javna nabava,https://www.foi.unizg.hr/sites/default/files/p...,/hr/dokument/popis-gospodarskih-subjekata-s-ko...,Popis gospodarskih subjekata s kojima se ne sm...
9,fv_zakljucci_2023-11-16_2._sjednica.pdf,https://www.foi.unizg.hr/sites/default/files/f...,16.11.2023,Zaključak,Fakultetsko vijeće,https://www.foi.unizg.hr/sites/default/files/f...,/hr/dokument/zakljucak-fakultetskog-vijeca-161...,Zaključak Fakultetskog vijeća 16.11.2023.


In [270]:
for link in df_test.pdf_link:
    download_pdf(path = './data/dummy_data', url = link)

Success: ./data/dummy_data/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf
Success: ./data/dummy_data/odluka-izvanredni_izbori_za_studentski_zbor-sijecanj_2024.pdf
Success: ./data/dummy_data/fv_dnevni_red_2023-12-07_3._sjednica.pdf
Success: ./data/dummy_data/politika-privatnosti-foi.pdf
Success: ./data/dummy_data/popis_gs_u_sukobu_interesa_2023.pdf
Success: ./data/dummy_data/fv_zakljucci_2023-11-16_2._sjednica.pdf
